# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [ ]:
!pip install --upgrade spacy==2.1.0 allennlp==0.9.0
import spacy

     |████████████████████████████████| 27.7MB 108kB/s 
     |████████████████████████████████| 7.6MB 29.6MB/s 
     |████████████████████████████████| 3.2MB 47.3MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 2.1MB 43.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 133kB 52.9MB/s 
     |████████████████████████████████| 133kB 54.9MB/s 
     |████████████████████████████████| 317kB 51.9MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 245kB 54.7MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 163kB 51.3MB/s 
     |████████████████████████████████| 5.6MB 48.0MB/s 
     |████████████████████████████████| 266kB 49.5MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 7.4MB 42.7MB/s 
     |████████████████████████████████| 1.2MB 50.6MB/s 
 

In [ ]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [ ]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [ ]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

from itertools import chain
vocab = Vocabulary.from_instances(chain(train_dataset, validation_dataset))

562it [00:00, 21001.60it/s]
23it [00:00, 14845.95it/s]
585it [00:00, 62155.94it/s]


## Define and Train Model

In [ ]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8133, loss: 1.0350 ||: 100%|██████████| 281/281 [00:01<00:00, 274.34it/s]
accuracy: 0.7878, loss: 1.2288 ||: 100%|██████████| 12/12 [00:00<00:00, 450.98it/s]
accuracy: 0.8442, loss: 0.7435 ||: 100%|██████████| 281/281 [00:01<00:00, 269.60it/s]
accuracy: 0.7878, loss: 1.2148 ||: 100%|██████████| 12/12 [00:00<00:00, 478.26it/s]
accuracy: 0.8442, loss: 0.7255 ||: 100%|██████████| 281/281 [00:01<00:00, 267.02it/s]
accuracy: 0.7878, loss: 1.1922 ||: 100%|██████████| 12/12 [00:00<00:00, 459.61it/s]
accuracy: 0.8442, loss: 0.7141 ||: 100%|██████████| 281/281 [00:01<00:00, 271.41it/s]
accuracy: 0.7878, loss: 1.1819 ||: 100%|██████████| 12/12 [00:00<00:00, 438.83it/s]
accuracy: 0.8442, loss: 0.7069 ||: 100%|██████████| 281/281 [00:01<00:00, 270.09it/s]
accuracy: 0.7878, loss: 1.1835 ||: 100%|██████████| 12/12 [00:00<00:00, 466.48it/s]
accuracy: 0.8442, loss: 0.7017 ||: 100%|██████████| 281/281 [00:01<00:00, 266.53it/s]
accuracy: 0.7878, loss: 1.1856 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8857142857142857,
 'best_validation_loss': 0.36948804757169756,
 'epoch': 99,
 'peak_cpu_memory_MB': 3231.968,
 'peak_gpu_0_memory_MB': 1060,
 'training_accuracy': 0.932634254845885,
 'training_cpu_memory_MB': 3231.968,
 'training_duration': '0:01:54.708402',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 1060,
 'training_loss': 0.17723424550791328,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8857142857142857,
 'validation_loss': 0.36948804757169756}

## Evaluation

The simple code below loops over the validation set, applying the model to each exmaple and collecting out the input token, gold-standard output, and model output. You can see from these methods how to access ground truth and model outputs for evaluation.

In [ ]:
def tag_sentence(s):
  tag_ids = np.argmax(model.forward_on_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
## Show the first example
baseline_output[0]

[(With, 'O', 'O'),
 (a, 'O', 'O'),
 (wave, 'O', 'O'),
 (of, 'O', 'O'),
 (his, 'O', 'O'),
 (hand, 'O', 'O'),
 (,, 'O', 'O'),
 (Peng, 'B-PERSON', 'B-PERSON'),
 (Dehuai, 'I-PERSON', 'I-PERSON'),
 (said, 'O', 'O'),
 (that, 'O', 'O'),
 (despite, 'O', 'O'),
 (being, 'O', 'O'),
 (over, 'O', 'O'),
 (100, 'B-CARDINAL', 'B-CARDINAL'),
 (regiments, 'O', 'O'),
 (,, 'O', 'O'),
 (let, 'O', 'O'),
 ('s, 'O', 'O'),
 (call, 'O', 'O'),
 (this, 'O', 'O'),
 (campaign, 'O', 'O'),
 (the, 'B-EVENT', 'O'),
 (Hundred, 'I-EVENT', 'I-EVENT'),
 (Regiments, 'I-EVENT', 'I-EVENT'),
 (Offensive, 'I-EVENT', 'I-EVENT'),
 (., 'O', 'O')]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [ ]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
    count = 0
    for i in tagged:
        prev_tag = ''
        for j in i:
            tag = j[2]
            if tag != 'O':
                if tag.startswith('I-'):
                    if prev_tag == '':
                        count += 1
                    else:
                        if tag[2:] != prev_tag:
                            count += 1
                    prev_tag = tag[2:]
                elif tag.startswith('B-'):
                    prev_tag = tag[2:]
            else:
                prev_tag = ''           
            
    return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# This is harsher than the token-level metric that the
# LSTM was trained to optimize, but it is the standard way
# of evaluating NER systems.
# Take tagger output as input
def span_stats(tagged):      
    idx_sen = 0
    span_mdl = []
    span_gld = []
    for t in tagged:
        idx_sen += 1
        idx_wrd = 0
        pretag_gld, pretag_mdl = '',''
        strt_gld, end_gld = 0,0
        strt_mdl, end_mdl = 0,0
        for w in t:
            idx_wrd += 1
            tag_gld = w[1]
            if tag_gld != 'O':
                if tag_gld.startswith("B-"):
                    end_gld = idx_wrd - 1
                    if pretag_gld != '':
                        span_gld.append((pretag_gld, strt_gld, end_gld, idx_sen))
                    strt_gld = idx_wrd
                    pretag_gld = tag_gld[2:]
            else: 
                end_gld = idx_wrd - 1
                if pretag_gld != '':
                    span_gld.append((pretag_gld, strt_gld, end_gld, idx_sen))
                pretag_gold = ''
                                         
            tag_mdl = w[2]
                                    
            if tag_mdl != 'O':
                if tag_mdl.startswith("B-"):
                    end_mdl = idx_wrd - 1
                    if pretag_mdl != '':
                        span_mdl.append((pretag_mdl, strt_mdl, end_mdl, idx_sen))
                    strt_mdl = idx_wrd
                    pretag_mdl = tag_mdl[2:]
                elif tag_mdl.startswith("I-"):
                    if pretag_mdl != tag_mdl[2:]:
                        pretag_mdl = ''
                        
            else:
                end_mdl = idx_wrd - 1
                if pretag_mdl != '':
                    span_mdl.append((pretag_mdl, strt_mdl, end_mdl, idx_sen))
                pretag_mdl = ''
        
        if pretag_gld != '':
            end_gld = idx_wrd - 1
            span_gld.append((pretag_gld, strt_gld, end_gld, idx_sen))
            
        if pretag_mdl != '':
            end_mdl = idx_wrd - 1
            span_mdl.append((pretag_mdl, strt_mdl, end_mdl, idx_sen))
       
    span_match = 0 
    for p in span_mdl:
        if p in span_gld:
            span_match += 1

    
    precision = span_match/ len(span_mdl)
    recall = span_match / len(span_gld)   
    f1 = 2*(precision * recall)/ (precision+recall)

    return {'precision': precision,
          'recall': recall,
          'f1': f1}
## You can check how many violations are made by the model output in predictor.
violations(baseline_output)

32

In [ ]:
span_stats(baseline_output)

{'f1': 0.08304498269896193,
 'precision': 0.5714285714285714,
 'recall': 0.04477611940298507}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `model` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the expression `model.forward_on_instance(s)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [ ]:
# This code shows how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [ ]:
n_inf = float("-inf")
transition_matrix = list()

initial_prob = []
vocab_dict = vocab.get_index_to_token_vocabulary('labels')
for ind in vocab_dict:
    label = vocab_dict[ind]
    if label == 'O' or label.startswith("B"):
        initial_prob.append(0)
    else:
        initial_prob.append(n_inf)

for row in vocab_dict:
    row_label = vocab_dict[row]
    cur_tag = ''
    if row_label != 'O':
        cur_tag = row_label[2:]
    new_list = list()
    for col in vocab_dict:
        col_label = vocab_dict[col]
        if row_label == 'O':
            if col_label =='O' or col_label.startswith("B"):
                new_list.append(0)
            else:
                new_list.append(n_inf)
        else:
            if col_label.startswith("B") or (col_label.startswith("I") and cur_tag == col_label[2:]) or col_label == 'O':
                new_list.append(0)
            else:
                new_list.append(n_inf)
    transition_matrix.append(new_list)
trans_prob = np.array(transition_matrix)

In [ ]:
def viterbi_decoder(s):
    obs_likelyhood = model.forward_on_instance(s)['tag_logits']
    emission_prob = np.array(obs_likelyhood)
    
    T = len(emission_prob)
    N = len(vocab_dict)
    
    viterabi = np.array([[0.0 for j in range(T)] for i in range(N)])
    backpointers = np.array([[-1 for j in range(T)] for i in range(N)])
    
    for i in range(T):
        if i==0:
            viterabi[:,0]= np.add(initial_prob, emission_prob[i])
        else:
            for j in range(N):
                viterabi[j,i]= np.max(np.add(trans_prob[:,j] , viterabi[:,i-1])) + emission_prob[i,j]
                backpointers[j,i] = np.argmax(np.add(trans_prob[:,j] , viterabi[:,i-1]))
    ind = np.argmax(viterabi[:,i])
    tag_ids = []
    while ind != -1:
        tag_ids.append(ind)
        ind = backpointers[ind,i]
        i -= 1
    tag_ids.reverse()
    fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    return list(fields)
    
    
baseline_viterbi_outputs = [viterbi_decoder(i) for i in validation_dataset]   

In [ ]:
violations(baseline_viterbi_outputs)

0

In [ ]:
span_stats(baseline_viterbi_outputs)

{'f1': 0.16560509554140126,
 'precision': 0.5652173913043478,
 'recall': 0.09701492537313433}